# Agent Over-Commit Test


## Setup


* `make start-proxy`
* `make start-rclone`
* `make start-agent`
* `make start-mlserver`

In [ ]:
%%bash
for i in {1..10}; 
do

echo "loading model iris$i"

data='{"request": { 
            "model":{ 
                "meta": {"name":"iris'"$i"'"}, 
                "modelSpec" : { 
                    "uri":"gs://seldon-models/sa/mlrepo/iris'"$i"'", 
                    "requirements":["sklearn"],  
                    "memoryBytes":65}, 
                "deploymentSpec": {"replicas":1} 
                } 
            },
            "version":1 
        }'

grpcurl -d "$data" \
-plaintext \
-import-path ../../apis/ \
-proto ../../apis/mlops/proxy/proxy.proto  0.0.0.0:8100 seldon.mlops.proxy.SchedulerProxy/LoadModel


done

In [ ]:
%%bash
grpcurl -d '{}' \
         -plaintext \
         -import-path ../../apis/ \
         -proto ../apis/mlops/agent_debug/agent_debug.proto  0.0.0.0:7777 seldon.mlops.agent_debug.AgentDebugService/ReplicaStatus

In [ ]:
%%bash
for i in {1..10}; 
do

time curl -v http://0.0.0.0:9999/v2/models/iris$i/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

done

In [ ]:
%%bash

grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -import-path ../../apis/mlops/v2_dataplane/ \
        -proto v2_dataplane.proto \
        -rpc-header seldon-model:iris2 \
        0.0.0.0:9998 inference.GRPCInferenceService/ModelInfer

In [ ]:
%%bash
for i in {1..10}; 
do 

for j in {1..10};
do
echo iris$j
time curl http://0.0.0.0:9999/v2/models/iris$j/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' &
done

done


In [ ]:
%%bash

for i in {3..8}; 
do

echo "loading model iris$i"

data='{"request": { 
            "model":{ 
                "meta": {"name":"iris'"$i"'"}, 
                "modelSpec" : { 
                    "uri":"gs://seldon-models/sa/mlrepo/iris'"$i"'", 
                    "requirements":["sklearn"],  
                    "memoryBytes":65}, 
                "deploymentSpec": {"replicas":1} 
                } 
            },
            "version":2 
        }'

grpcurl -d "$data" \
-plaintext \
-import-path ../../apis/ \
-proto ../../apis/mlops/proxy/proxy.proto  0.0.0.0:8100 seldon.mlops.proxy.SchedulerProxy/LoadModel

done

In [ ]:
!grpcurl -d '{"model": {"name" : "iris4"}}' \
         -plaintext \
         -import-path ../../apis/ \
         -proto ../../apis/mlops/proxy/proxy.proto  0.0.0.0:8100 seldon.mlops.proxy.SchedulerProxy/UnloadModel